In [11]:

%load_ext autoreload
%autoreload 2

from IPython import display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:

import zipfile
import os
import google.auth
import pandas as pd

from tqdm import tqdm
from google.cloud import bigquery
from pprint import pprint
from datetime import datetime
from zipfile import ZipFile
from google.cloud import bigquery

from mailer import Mailer
from queries_dict import names_queries
from read_secret import read_secret

In [13]:
datetime.today() 

datetime.datetime(2022, 6, 2, 16, 37, 5, 933754)

In [14]:
retrieve_dict = lambda country: {k:v for k, v in names_queries.items() if k.endswith(country)}
d_col = retrieve_dict("col")
d_mx = retrieve_dict("mx")

In [15]:

def run_queries(names_queries, client):
    d = dict()
    n = len(names_queries)
    pbar = tqdm(names_queries.items(), total=n, position=0)
    for name, query in pbar:
        try:
            query_job = client.query(query)
            df = query_job.to_dataframe()
            d[name] = df 
        except Exception as e:
            print('[!]' + str(e) + name)
            d[name] = pd.DataFrame()
    return d
    

In [16]:


credentials, project = google.auth.default(
     scopes=[
        "https://www.googleapis.com/auth/cloud-platform",
         "https://www.googleapis.com/auth/drive",
         "https://www.googleapis.com/auth/bigquery",
     ]
)

# client = bigquery.Client(project="papyrus-data")
client = bigquery.Client(credentials=credentials, project=project)

In [17]:
print("fetching col rows")
res_col = run_queries(d_col, client)
print("fetching mex rows")
res_mex = run_queries(d_mx, client)

fetching col rows


100%|██████████| 5/5 [00:57<00:00, 11.44s/it]


fetching mex rows


100%|██████████| 5/5 [00:14<00:00,  2.85s/it]


In [18]:
# credentials, project = google.auth.default(
#      scopes=[
#         "https://www.googleapis.com/auth/cloud-platform",
#          "https://www.googleapis.com/auth/drive",
#          "https://www.googleapis.com/auth/bigquery",
#      ]
# )
# # client = bigquery.Client(project="papyrus-data")
# client = bigquery.Client(credentials=credentials, project=project)
# query = d_col['calificados_fuente_col']
# job = client.query(query)
# job.to_dataframe()

In [19]:
d = {**res_col, **res_mex}
errors = []

for f in os.listdir("csv"):
    os.remove(os.path.join("csv", f))
os.rmdir("csv")
os.mkdir("csv")

for k, v in d.items():
    if v.empty:
        errors.append(k)
    else:
        print(k, v.shape)
        fname = f"csv/{k}.csv"
        v.to_csv(fname)


with ZipFile("csv.zip", "w", zipfile.ZIP_DEFLATED) as fp:
    for file in os.listdir("csv"):
        fp.write(f"csv/{file}")

cierres_col (3126, 5)
calificados_fuente_col (438576, 4)
calificados_totales_col (1112, 4)
fuentes_web_col (143964, 4)
ventanas_col (1382, 4)
ventanas_mx (119458, 5)
calificados_fuente_web_mx (271, 5)
calificados_todas_las_fuentes_mx (271, 5)
fuente_web_mx (32667, 5)
cierres_mx (400, 5)


In [20]:
MY_MAIL = 'angel.manriquez.habi@gmail.com'
MY_PASSWD = read_secret("papyrus-data-mx", "papyrus-data-mx-angel-marketing")
target_mail = "angellopez@tuhabi.mx"

In [21]:

mailer = Mailer(MY_MAIL, MY_PASSWD)
mailer.sendMessage(target_mail, ['csv.zip'], "Aqui tienes las consultas del mes.", "automatizacion envio de consultas")

100%|██████████| 1/1 [00:00<00:00,  3.73it/s]


True

In [22]:
if errors:
    with open("error.log", 'w') as fp:
        for e in errors:
            fp.write(e + "\n")

    mailer.sendMessage("angellopez@tuhabi.mx", ["error.log"], "Listado de consultas con observaciones.", "Errores marketing automatizacion")